<a href="https://colab.research.google.com/github/jinbaaaaaang/oss-siot/blob/main/GPU_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content

/content


In [ ]:
!git clone https://github.com/jinbaaaaaang/siot.git
%cd /content/siot/backend

In [3]:
import os
os.environ['GOOGLE_TRANSLATE_API_KEY'] = ''
os.environ['GEMINI_API_KEY']=''

In [4]:
os.environ["GOOGLE_CLOUD_PROJECT_ID"] = ""
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ""

In [ ]:
# ===== bitsandbytes 및 필수 패키지 설치 =====
print(" 패키지 설치 중...")

# 기존 설치 제거
!pip uninstall -y bitsandbytes 2>/dev/null || true

# PyTorch 설치 (CUDA 11.8)
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# bitsandbytes 설치
!pip install -q bitsandbytes

# 다른 필수 패키지
!pip install -q transformers accelerate

# 설치 확인
try:
    import bitsandbytes as bnb
    print("✅ bitsandbytes 설치 완료")
    print(f"   버전: {bnb.__version__}")
except Exception as e:
    print(f"❌ bitsandbytes 설치 실패: {e}")
    print("💡 런타임을 재시작한 후 다시 시도하세요")

In [ ]:
# 1) 혹시 떠 있는 uvicorn 정리
!pkill -f uvicorn || echo "no uvicorn"
!ps aux | grep uvicorn

# 2) 백엔드 디렉토리 이동
%cd /content/siot-OSS/backend
# !python -m uvicorn app.main:app --host 0.0.0.0 --port 8000

# 3) uvicorn 백그라운드 실행
!nohup uvicorn app.main:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &

In [ ]:
# 4) 서버 살아있는지 health 체크
!curl -v http://localhost:8000/health
!ps aux | grep uvicorn

In [ ]:
!pip install pyngrok --quiet

from pyngrok import ngrok

# 혹시 기존 터널 있으면 정리
ngrok.kill()

NGROK_TOKEN = "" # 토큰 넣기

ngrok.set_auth_token(NGROK_TOKEN)

public_url = ngrok.connect(8000, bind_tls=True)
print("🔗 ngrok public URL:", public_url)
print("   health:", f"{public_url}/health")